# Building a RAG application from scratch

Here is a high-level overview of the system we want to build:

<img src='images/system1.png' width="1200">

Let's start by loading the environment variables we need to use.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY environment variable is not set. Please check your .env file.")

if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY environment variable is not set. Please check your .env file.")

# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

### Setting up the `.env` file

Ensure you have a `.env` file in the root directory of your project with the following content:

```plaintext
OPENAI_API_KEY=your_openai_api_key_here
```

Replace `your_openai_api_key_here` with your actual OpenAI API key.

## Setting up the model
Let's define the LLM model that we'll use as part of the workflow.

In [3]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")


We can test the model by asking a simple question.

In [128]:
model.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

AIMessage(content='The Los Angeles Dodgers won the 2020 World Series during the COVID-19 pandemic.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 21, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLJwJ5haJ6DMvsWvcN5xG01lvRMHO', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4be6b64-7eec-467c-8c6e-f555eae32771-0', usage_metadata={'input_tokens': 21, 'output_tokens': 19, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

The result from the model is an `AIMessage` instance containing the answer. We can extract this answer by chaining the model with an [output parser](https://python.langchain.com/docs/how_to/#output-parsers).

Here is what chaining the model with an output parser looks like:

<img src='images/chain1.png' width="1200">

For this example, we'll use a simple `StrOutputParser` to extract the answer as a string.

In [129]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

'The Los Angeles Dodgers won the World Series during the COVID-19 pandemic in 2020. They defeated the Tampa Bay Rays in the World Series to claim their first championship since 1988.'

## Introducing prompt templates

We want to provide the model with some context and the question. [Prompt templates](https://python.langchain.com/docs/how_to/#prompt-templates) are a simple way to define and reuse prompts.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")

NameError: name 'prompt' is not defined

We can now chain the prompt with the model and the output parser.

<img src='images/chain2.png' width="1200">

In [131]:
chain = prompt | model | parser
chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

'Susana'

## Combining chains

We can combine different chains to create more complex workflows. For example, let's create a second chain that translates the answer from the first chain into a different language.

Let's start by creating a new prompt template for the translation chain:

In [132]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

We can now create a new translation chain that combines the result from the first chain with the translation prompt.

Here is what the new workflow looks like:

<img src='images/chain3.png' width="1200">

In [133]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Mary's sister is Susana. She doesn't have any more siblings.",
        "question": "How many sisters does Mary have?",
        "language": "Spanish",
    }
)

'María tiene una hermana.'

## Transcribing the YouTube Video

The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using [OpenAI's Whisper](https://openai.com/research/whisper).

In [134]:
import tempfile
import whisper
from pytube import YouTube


# Let's do this only if we haven't created the transcription file yet.
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as file:
            file.write(transcription)

Let's read the transcription and display the first few characters to ensure everything works as expected.

In [8]:
with open("transcripts/Lemonade Stand Podcast Combined.txt") as file:
    transcription = file.read()

transcription[:1000]

"You know, I'll skid you a structuring guidance. This show this show. Let's move on to flamma guys. It's time for Doug Doug to call out Mark Rober. Yeah, you need to do it. We need a personal. We need an enemy for this podcast. Yeah, all good podcasts of enemies. Did you know that if you go to x.com, the everything app and you submit the winning college basketball bracket for March madness, you could win a trip to Mars. And this is a real thing that's on Twitter. And so what I'm thinking is because we have a big audience now. If every one of our viewers, we also met a different bracket, but we put H. Rock's name than one of us will win and then Elon will kidnap him and say to Mars. And we'll finally be really you. Is that what you're thinking? I'm digging after our episode last week where you defended Elon Musk. He reached out and hired you as an ad man to promote his. It's like, don't worry. Whatever is submitted. He will win the contest. I do like that. It says it says win a free tri

## Using the entire transcription as context

If we try to invoke the chain using the transcription as context, the model will return an error because the context is too long.

Large Language Models support limitted context sizes. The video we are using is too long for the model to handle, so we need to find a different solution.

In [9]:
try:
    chain.invoke({
        "context": transcription,
        "question": "Is reading papers a good idea?"
    })
except Exception as e:
    print(e)

name 'chain' is not defined


## Splitting the transcription

Since we can't use the entire transcription as the context for the model, a potential solution is to split the transcription into smaller chunks. We can then invoke the model using only the relevant chunks to answer a particular question:

<img src='images/system2.png' width="1200">

Let's start by loading the transcription in memory:

In [11]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcripts/Lemonade Stand Podcast Combined.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'transcripts/Lemonade Stand Podcast Combined.txt'}, page_content="You know, I'll skid you a structuring guidance. This show this show. Let's move on to flamma guys. It's time for Doug Doug to call out Mark Rober. Yeah, you need to do it. We need a personal. We need an enemy for this podcast. Yeah, all good podcasts of enemies. Did you know that if you go to x.com, the everything app and you submit the winning college basketball bracket for March madness, you could win a trip to Mars. And this is a real thing that's on Twitter. And so what I'm thinking is because we have a big audience now. If every one of our viewers, we also met a different bracket, but we put H. Rock's name than one of us will win and then Elon will kidnap him and say to Mars. And we'll finally be really you. Is that what you're thinking? I'm digging after our episode last week where you defended Elon Musk. He reached out and hired you as an ad man to promote his. It's like, don't worry.

There are many different ways to split a document. For this example, we'll use a simple splitter that splits the document into chunks of a fixed size. Check [Text Splitters](https://python.langchain.com/docs/how_to/#text-splitters) for more information about different approaches to splitting documents.

For illustration purposes, let's split the transcription into chunks of 100 characters with an overlap of 20 characters and display the first few chunks:

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(metadata={'source': 'transcripts/Lemonade Stand Podcast Combined.txt'}, page_content="You know, I'll skid you a structuring guidance. This show this show. Let's move on to flamma guys."),
 Document(metadata={'source': 'transcripts/Lemonade Stand Podcast Combined.txt'}, page_content="on to flamma guys. It's time for Doug Doug to call out Mark Rober. Yeah, you need to do it. We need"),
 Document(metadata={'source': 'transcripts/Lemonade Stand Podcast Combined.txt'}, page_content='to do it. We need a personal. We need an enemy for this podcast. Yeah, all good podcasts of'),
 Document(metadata={'source': 'transcripts/Lemonade Stand Podcast Combined.txt'}, page_content='good podcasts of enemies. Did you know that if you go to x.com, the everything app and you submit'),
 Document(metadata={'source': 'transcripts/Lemonade Stand Podcast Combined.txt'}, page_content='app and you submit the winning college basketball bracket for March madness, you could win a trip')]

For our specific application, let's use 1000 characters instead:

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## Finding the relevant chunks

Given a particular question, we need to find the relevant chunks from the transcription to send to the model. Here is where the idea of **embeddings** comes into play.

An embedding is a mathematical representation of the semantic meaning of a word, sentence, or document. It's a projection of a concept in a high-dimensional space. Embeddings have a simple characteristic: The projection of related concepts will be close to each other, while concepts with different meanings will lie far away. You can use the [Cohere's Embed Playground](https://dashboard.cohere.com/playground/embed) to visualize embeddings in two dimensions.

To provide with the most relevant chunks, we can use the embeddings of the question and the chunks of the transcription to compute the similarity between them. We can then select the chunks with the highest similarity to the question and use them as the context for the model:

<img src='images/system3.png' width="1200">

Let's generate embeddings for an arbitrary query:

In [14]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Who is Mary's sister?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[-0.00137994228862226, -0.03450090438127518, -0.011502386070787907, 0.001241557183675468, -0.02611961029469967, 0.009081818163394928, -0.015649249777197838, 0.0017278597224503756, -0.011827629990875721, -0.03319992497563362]


To illustrate how embeddings work, let's first generate the embeddings for two different sentences:

In [ ]:
sentence1 = embeddings.embed_query("Mary's sister is Susana")
sentence2 = embeddings.embed_query("Pedro's mother is a teacher")

We can now compute the similarity between the query and each of the two sentences. The closer the embeddings are, the more similar the sentences will be.

We can use [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) to calculate the similarity between the query and each of the sentences:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9173235346162657, 0.7679756802174771)

## Setting up a Vector Store

We need an efficient way to store document chunks, their embeddings, and perform similarity searches at scale. To do this, we'll use a **vector store**.

A vector store is a database of embeddings that specializes in fast similarity searches. 

<img src='images/system4.png' width="1200">

To understand how a vector store works, let's create one in memory and add a few embeddings to it:

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Mary's sister is Susana",
        "John and Tommy are brothers",
        "Patricia likes white cars",
        "Pedro's mother is a teacher",
        "Lucia drives an Audi",
        "Mary has two siblings",
    ],
    embedding=embeddings,
)

We can now query the vector store to find the most similar embeddings to a given query:

In [ ]:
vectorstore1.similarity_search_with_score(query="Who is Mary's sister?", k=3)

[(Document(metadata={}, page_content="Mary's sister is Susana"),
  0.9173235428839123),
 (Document(metadata={}, page_content='Mary has two siblings'),
  0.9045029978848256),
 (Document(metadata={}, page_content='John and Tommy are brothers'),
  0.8013182122337675)]

## Connecting the vector store to the chain

We can use the vector store to find the most relevant chunks from the transcription to send to the model. Here is how we can connect the vector store to the chain:

<img src='images/chain4.png' width="1200">

We need to configure a [Retriever](https://python.langchain.com/docs/how_to/#retrievers). The retriever will run a similarity search in the vector store and return the most similar documents back to the next step in the chain.

We can get a retriever directly from the vector store we created before: 

In [ ]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Who is Mary's sister?")

[Document(metadata={}, page_content="Mary's sister is Susana"),
 Document(metadata={}, page_content='Mary has two siblings'),
 Document(metadata={}, page_content='John and Tommy are brothers'),
 Document(metadata={}, page_content="Pedro's mother is a teacher")]

Our prompt expects two parameters, "context" and "question." We can use the retriever to find the chunks we'll use as the context to answer the question.

We can create a map with the two inputs by using the [`RunnableParallel`](https://python.langchain.com/docs/how_to/parallel/) and [`RunnablePassthrough`](https://python.langchain.com/docs/how_to/passthrough/) classes. This will allow us to pass the context and question to the prompt as a map with the keys "context" and "question."

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("What color is Patricia's car?")

{'context': [Document(metadata={}, page_content='Patricia likes white cars'),
  Document(metadata={}, page_content='Lucia drives an Audi'),
  Document(metadata={}, page_content="Pedro's mother is a teacher"),
  Document(metadata={}, page_content="Mary's sister is Susana")],
 'question': "What color is Patricia's car?"}

Let's now add the setup map to the chain and run it:



In [ ]:
chain = setup | prompt | model | parser
chain.invoke("What color is Patricia's car?")

'White'

Let's invoke the chain using another example:

In [ ]:
chain.invoke("What car does Lucia drive?")

'Lucia drives an Audi.'

## Loading transcription into the vector store

We initialized the vector store with a few random strings. Let's create a new vector store using the chunks from the video transcription.

In [ ]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

Let's set up a new chain using the correct vector store. This time we are using a different equivalent syntax to specify the [`RunnableParallel`](https://python.langchain.com/docs/how_to/parallel/) portion of the chain:

In [ ]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence is described as the next stage of development in artificial intelligence. It involves the creation of artificial intelligences that are capable of uncovering and solving complex puzzles or problems in the universe.'

## Setting up Pinecone

So far we've used an in-memory vector store. In practice, we need a vector store that can handle large amounts of data and perform similarity searches at scale. For this example, we'll use [Pinecone](https://www.pinecone.io/).

The first step is to create a Pinecone account, set up an index, get an API key, and set it as an environment variable `PINECONE_API_KEY`.

Then, we can load the transcription documents into Pinecone:

In [ ]:
from langchain_pinecone import PineconeVectorStore

index_name = "youtube-index"

pinecone_vectorstore = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)


NameError: name 'documents' is not defined

Let's now run a similarity search on a pinecone to make sure everything works:

In [ ]:
# Perform a similarity search using the Pinecone vector store
pinecone_vectorstore.similarity_search("What is Hollywood going to start doing?", k=3)

[Document(id='d764898e-4f5b-481f-a479-3c149bfe6be4', metadata={'source': 'transcription.txt'}, page_content="It's like high quality audio and you're speaking usually pretty clearly. I don't know what open AI's plans are either. Yeah, there's always fun projects basically. And stable diffusion also is opening up a huge amount of experimentation. I would say in the visual realm and generating images and videos and movies. I'll think like videos now. And so that's going to be pretty crazy. That's going to almost certainly work and it's going to be really interesting when the cost of content creation is going to fall to zero. You used to need a painter for a few months to paint a thing and now it's going to be speak to your phone to get your video. So Hollywood will start using it to generate scenes, which completely opens up. Yeah, so you can make a movie like Avatar eventually for under a million dollars. Much less. Maybe just by talking to your phone. I mean, I know it sounds kind of cr

Let's setup the new chain using Pinecone as the vector store:

In [ ]:
chain = (
    {"context": pinecone_vectorstore.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("What is this podcast about?")

'The podcast seems to be about artificial intelligence, machine learning, and the work of Andre Karpathy.'